In [ ]:
import random
import os

def generate_random_bits(num_bits, file_path):
    # Generate random bits
    bits = ''.join(str(random.randint(0, 1)) for _ in range(num_bits))
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    # Write bits to the file
    with open(file_path, 'w') as file:
        file.write(bits)

# Define the number of bits and file path
num_bits = 20000000  # Change this to the number of bits you need
file_path = r'C:\Users\Hendy Group\OneDrive\Desktop\final_graduation_project\folder00\random_bits.txt'

# Call the function
generate_random_bits(num_bits, file_path)


In [ ]:
def bin_to_text_binary(bin_file_path, text_file_path):
    """
    Converts a binary file to a text file with binary (0s and 1s) representation.

    :param bin_file_path: Path to the input binary file.
    :param text_file_path: Path to the output text file.
    """
    try:
        with open(bin_file_path, 'rb') as bin_file:
            binary_content = bin_file.read()
        
        # Convert binary content to a string of 0s and 1s
        binary_string = ''.join(format(byte, '08b') for byte in binary_content)

        with open(text_file_path, 'w') as text_file:
            text_file.write(binary_string)

        print(f"Successfully converted {bin_file_path} to {text_file_path}")

    except Exception as e:
        print(f"Error: {e}")

# Define paths
bin_file_path = r"C:\Users\Hendy Group\OneDrive\Desktop\final_graduation_project\folder00\output_binary_file.bin"
text_file_path = r"C:\Users\Hendy Group\OneDrive\Desktop\final_graduation_project\folder00\input_bits.txt"

# Convert the binary file to a text file with binary representation
bin_to_text_binary(bin_file_path, text_file_path)


In [ ]:
def text_to_bin(text_file_path, bin_file_path):
    try:
        with open(text_file_path, 'r') as text_file:
            binary_string = text_file.read().strip()
        
        # Ensure the binary string length is a multiple of 8
        if len(binary_string) % 8 != 0:
            raise ValueError("The binary string length must be a multiple of 8.")

        # Convert the binary string to bytes
        byte_array = bytearray(int(binary_string[i:i+8], 2) for i in range(0, len(binary_string), 8))

        with open(bin_file_path, 'wb') as bin_file:
            bin_file.write(byte_array)

        print(f"Successfully converted {text_file_path} to {bin_file_path}")

    except Exception as e:
        print(f"Error: {e}")

# Define paths
text_file_path = r'C:\Users\Hendy Group\OneDrive\Desktop\final_graduation_project\folder00\extracted_message.txt'
bin_file_path = r"C:\Users\Hendy Group\OneDrive\Desktop\final_graduation_project\restored_input.bin"

# Convert the text file with binary representation to a binary file
text_to_bin(text_file_path, bin_file_path)


In [ ]:
import cv2
import numpy as np
import os

# Function to hide message in the last 4 bits of image pixels
def hide_message_4bits(image, binary_message):
    # Flatten image pixel values
    flattened_image = image.reshape((-1, 3))

    # Process 4 bits at a time
    for i in range(0, len(binary_message), 4):
        # Get the next 4 bits of the message
        bits = binary_message[i:i + 4]
        bits = bits.ljust(4, '0')  # Pad with zeros if less than 4 bits

        # Convert bits to integer
        bits_int = int(bits, 2)

        # Embed 4 bits into the last 4 bits of the current pixel's blue channel
        flattened_image[i // 4][-1] = (flattened_image[i // 4][-1] & 240) | bits_int
    
    # Reshape the image back to original shape
    modified_image = flattened_image.reshape(image.shape)

    return modified_image

# Function to extract message hidden in the last 4 bits of image pixels
def extract_message_4bits(image, message_length):
    # Flatten image pixel values
    flattened_image = image.reshape((-1, 3))

    # Extract 4 bits from each pixel
    extracted_message = []
    for i in range(message_length // 4):
        bits_int = flattened_image[i][-1] & 15
        bits = format(bits_int, '04b')
        extracted_message.append(bits)
    
    # Join the bits into a single binary string
    binary_message = ''.join(extracted_message)

    return binary_message

# Function to write binary message to a text file
def write_binary_message_to_file(binary_message, file_path):
    with open(file_path, 'w') as file:
        file.write(binary_message)

# Function to calculate SNR between two images
def calculate_snr(original_image, modified_image):
    # Convert images to float64 to avoid overflow during computations
    original_image = original_image.astype(np.float64)
    modified_image = modified_image.astype(np.float64)
    
    # Calculate signal power
    signal_power = np.sum(original_image ** 2)
    
    # Calculate noise power as the sum of squared differences
    noise = original_image - modified_image
    noise_power = np.sum(noise ** 2)
    
    # Calculate SNR in decibels (dB)
    snr = 10 * np.log10(signal_power / noise_power)
    
    return snr

# Function to calculate maximum embedding capacity of the cover image
def calculate_max_capacity(image):
    height, width, channels = image.shape
    
    # Calculate the total number of bits that can be hidden
    max_capacity_bits = height * width * channels * 4
    
    return max_capacity_bits

# Paths for input files and directories
binary_message_path = r'C:\Users\Hendy Group\OneDrive\Desktop\final_graduation_project\folder00\random_bits.txt'
cover_image_dir = r'C:\Users\Hendy Group\OneDrive\Desktop\final_graduation_project\cover_image'
output_dir = r'C:\Users\Hendy Group\OneDrive\Desktop\final_graduation_project\folder00'

# Read binary data from text file
with open(binary_message_path, 'r') as file:
    binary_message = file.read()

# Determine the number of cover images needed
cover_image_files = sorted([os.path.join(cover_image_dir, f) for f in os.listdir(cover_image_dir) if f.endswith('.jpg')])
num_cover_images = len(cover_image_files)

# Load the first cover image to get its dimensions
cover_image = cv2.imread(cover_image_files[0])
cover_image_rgb = cv2.cvtColor(cover_image, cv2.COLOR_BGR2RGB)
max_capacity = calculate_max_capacity(cover_image_rgb)

# Split binary message into chunks
chunk_size = max_capacity // 4
binary_message_chunks = [binary_message[i:i + chunk_size] for i in range(0, len(binary_message), chunk_size)]
num_full_chunks = len(binary_message_chunks)

# Embed each chunk into a separate cover image
modified_images = []
for j, chunk in enumerate(binary_message_chunks):
    # Determine the current cover image index using modulo operation
    i = j % num_cover_images
    
    # Load the current cover image
    cover_image = cv2.imread(cover_image_files[i])
    cover_image_rgb = cv2.cvtColor(cover_image, cv2.COLOR_BGR2RGB)
    
    # Embed the message chunk into the current cover image
    modified_image = hide_message_4bits(cover_image_rgb, chunk)
    modified_images.append(modified_image)
    
    # Save the modified image with a sequential name
    output_image_path = os.path.join(output_dir, f'{j + 1}.jpg')
    cv2.imwrite(output_image_path, cv2.cvtColor(modified_image, cv2.COLOR_RGB2BGR))
    print(f"Modified image {j + 1} saved to: {output_image_path}")

# Extract hidden message from each modified image
extracted_messages = []
for j, modified_image in enumerate(modified_images):
    # Determine the current cover image index using modulo operation
    i = j % num_cover_images
    
    chunk_size = len(binary_message_chunks[j])
    extracted_message = extract_message_4bits(modified_image, chunk_size)
    extracted_messages.append(extracted_message)

# Combine all extracted chunks into the complete binary message
extracted_message_4bits = ''.join(extracted_messages)

# Write extracted binary message to a text file
extracted_message_path = os.path.join(output_dir, 'extracted_message.txt')
write_binary_message_to_file(extracted_message_4bits, extracted_message_path)
print(f"Extracted binary message saved to: {extracted_message_path}")

# Calculate SNR for each encoded image
snrs = []
for j in range(len(modified_images)):
    # Determine the current cover image index using modulo operation
    i = j % num_cover_images
    
    cover_image = cv2.imread(cover_image_files[i])
    cover_image_rgb = cv2.cvtColor(cover_image, cv2.COLOR_BGR2RGB)
    
    snr = calculate_snr(cover_image_rgb.astype(np.float64), modified_images[j].astype(np.float64))
    snrs.append(snr)
    print(f"SNR (4 bits) for Image {j + 1}: {snr:.2f} dB")

# Calculate average SNR
average_snr = np.mean(snrs)
print(f"Average SNR (4 bits) across all images: {average_snr:.2f} dB")

# Calculate maximum embedding capacity of the cover image
print(f"Maximum embedding capacity of a cover image: {max_capacity} bits")


In [ ]:
import cv2
import os

def images_to_video(images_dir, output_video_path, fps=12, image_size=(1024, 1024)):
    # Get sorted list of image files based on numerical order
    image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.jpg')], key=lambda x: int(os.path.splitext(x)[0]))

    if not image_files:
        print("No images found in the directory.")
        return

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can change the codec as needed
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, image_size)

    for image_file in image_files:
        image_path = os.path.join(images_dir, image_file)
        image = cv2.imread(image_path)
        resized_image = cv2.resize(image, image_size)
        video_writer.write(resized_image)

    video_writer.release()
    print(f"Video saved to '{output_video_path}'")

# Example usage
images_dir = r'C:\Users\Hendy Group\OneDrive\Desktop\final_graduation_project\folder00'
output_video_path = r'C:\Users\Hendy Group\OneDrive\Desktop\final_graduation_project\folder00\output_video.mp4'

images_to_video(images_dir, output_video_path)


In [ ]:
import cv2
import os

def video_to_images(video_path, output_dir, image_size=(1024, 1024)):
    # Open the video file
    video_capture = cv2.VideoCapture(video_path)
    if not video_capture.isOpened():
        print(f"Error opening video file {video_path}.")
        return

    # Create output directory within the same directory as the video
    video_directory = os.path.dirname(video_path)
    output_images_dir = os.path.join(video_directory, output_dir)
    os.makedirs(output_images_dir, exist_ok=True)

    frame_index = 0
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        resized_frame = cv2.resize(frame, image_size)
        image_path = os.path.join(output_images_dir, f'image_{frame_index:04d}.jpg')
        cv2.imwrite(image_path, resized_frame)
        frame_index += 1

    video_capture.release()
    print(f"{frame_index} images saved to '{output_images_dir}'")

# Example usage
video_path =  r'C:\Users\Hendy Group\OneDrive\Desktop\final_graduation_project\folder00\output_video.mp4'
output_images_dir = 'extracted_images2'  # Relative to the video directory

video_to_images(video_path, output_images_dir)
